In [12]:
import cv2
import numpy as np
def preprocess_captcha(image_path, character_color):
    # Load the captcha image
    captcha = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(captcha, cv2.COLOR_BGR2GRAY)

    # Create a mask for the character color region
    mask = cv2.inRange(gray, character_color, character_color)

    # Set the character region to black
    captcha[mask > 0] = (0, 0, 0)  # Set all channels to black (0, 0, 0)

    # Set the background region to white
    captcha[mask == 0] = (255, 255, 255)  # Set all channels to white (255, 255, 255)

    return captcha

# Example usage
image_path = 'captchas/bdHBnp.png'
character_color = 102  # Adjust this value to match the character color (hex: #666666)

result = preprocess_captcha(image_path, character_color)

# Display the preprocessed image
cv2.imshow("Preprocessed Image", result)
cv2.waitKey(0)


-1

In [16]:
# To make sure that i have collected all the characters.
import os

chars = {}
letters = "abcdefghijklmnopqrstuvwxyz"
for i in letters:
    chars[i] = 0

for i in letters.upper():
    chars[i] = 0

for i in range(10):
    chars[str(i)] = 0

image_names = os.listdir("./captchas")

for image_name in image_names:
    characters = image_name[:-4]
    if len(characters)==6:
        for c in characters:
            chars[c] += 1

for i in chars:
    if chars[i]==0:
        print(i)

print("I guess these characters never appear, as it is ambiguous")

i
l
o
q
I
O
Q
0
1
I guess these characters never appear, as it is ambiguous


In [17]:
"0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ".find("z")

35

In [10]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the input shape of your captcha images
input_shape = (160, 75, 1)  # Assuming grayscale captcha images, adjust the channel value if using RGB images

# Define the character set
characters = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
num_characters = len(characters)

# Define the maximum length of the captcha sequence
max_sequence_length = 6

# Define the scale factor for mapping output values to the desired range
scale_factor = num_characters - 1  # Scale factor to map values from 0 to 1 to the range of 0 to num_characters-1

# Define the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(max_sequence_length, activation='relu')
])

# Scale the output values to the desired range
model.add(layers.Lambda(lambda x: scale_factor * tf.nn.relu(x)))

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 158, 73, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 79, 36, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 77, 34, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 38, 17, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 41344)             0         
                                                                 
 dense (Dense)               (None, 128)               5

In [78]:
# Augmentation with preprocessing images
import os
import cv2
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the paths for the input and output directories
input_dir = './captchas'
output_dir = './preprocessed_captchas'

# Set up the data augmentation parameters
datagen = ImageDataGenerator(
    rotation_range=10,  # Random rotation between -10 and +10 degrees
    width_shift_range=0.1,  # Random horizontal shift by 10% of the image width
    height_shift_range=0.1,  # Random vertical shift by 10% of the image height
    shear_range=0.2,  # Apply shear transformation with a maximum shear of 20%
    zoom_range=0.2,  # Randomly zoom in or out by 20%
    fill_mode='constant',  # Fill any empty pixels with constant value (e.g., black)
    cval=255  # Set the constant value to white (255)
)

# Iterate over the images in the input directory
for filename in os.listdir(input_dir):
    # Read the image
    image_path = os.path.join(input_dir, filename)
    image = cv2.imread(image_path)
    
    # Preprocess the image
    preprocessed_image = preprocess_captcha(image_path, character_color)
    
    # Expand dimensions to match the expected input shape of the model
    # preprocessed_image = np.expand_dims(preprocessed_image, axis=-1)
    
    # Create a directory for the current image in the output directory
    output_subdir = os.path.join(output_dir, os.path.splitext(filename)[0])
    if not os.path.exists(output_subdir):
        os.makedirs(output_subdir)
    
    # Generate augmented images and save them in the output directory
    for i, batch in enumerate(datagen.flow(np.array([preprocessed_image]), batch_size=1)):
        augmented_image = batch[0].astype(np.uint8)
        
        # Save the augmented image
        output_filename = f'{os.path.splitext(filename)[0]}_{i}.png'
        output_path = os.path.join(output_subdir, output_filename)
        cv2.imwrite(output_path, augmented_image)
        
        # Break the loop after generating a desired number of augmented images
        if i == 99:
            break


In [68]:
import tensorflow as tf
from tensorflow.keras import layers

# Define the input shape of your captcha images
input_shape = (160, 75, 1)  # Assuming grayscale captcha images, adjust the channel value if using RGB images

# Define the character set
characters = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
num_characters = len(characters)

# Define the maximum length of the captcha sequence
max_sequence_length = 6

# Define the scale factor for mapping output values to the desired range
scale_factor = num_characters - 1  # Scale factor to map values from 0 to 1 to the range of 0 to num_characters-1

# Define the CNN model
model = tf.keras.Sequential([
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(max_sequence_length * num_characters, activation='relu'),
    layers.Reshape((max_sequence_length, num_characters))
])

# Scale the output values to the desired range
model.add(layers.Lambda(lambda x: scale_factor * tf.nn.relu(x)))

# Compile the model
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Print the model summary
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 158, 73, 32)       320       
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 79, 36, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 77, 34, 64)        18496     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 38, 17, 64)       0         
 2D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 41344)             0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [102]:
from tensorflow import keras
from tensorflow.keras import layers

# Set the paths for the preprocessed images
images_dir = 'preprocessed_captchas'


# Define the characters for mapping labels
characters = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"


# Function to convert labels to list form
def convert_label_to_list(label):
    label_list = []
    for char in label:
        index = characters.index(char)
        oh = [0]*62
        oh[index] = 1
        label_list.append(oh)
    return label_list

# Load the preprocessed images and labels
X = []
y = []
for subdir, dirs, files in os.walk(images_dir):
    for file in files:
        # Load the image
        image_path = os.path.join(subdir, file)
        image = keras.preprocessing.image.load_img(image_path, color_mode='grayscale', target_size=input_shape[:2])
        image = keras.preprocessing.image.img_to_array(image)
        image /= 255.0
        X.append(image)
        
        # Extract the label from the file name
        label = file.split('.')[0][:6]
        label_list = convert_label_to_list(label)
        y.append(label_list)

# Convert the lists to numpy arrays
X = np.array(X)
y = np.array(y)


In [103]:
# Split the data into training and validation sets
validation_split = 0.2
split_index = int(len(X) * (1 - validation_split))
X_train, X_val = X[:split_index], X[split_index:]
y_train, y_val = y[:split_index], y[split_index:]

# Convert the labels to one-hot encoding
num_classes = len(characters)
# y_train = keras.utils.to_categorical(y_train, num_classes)
# y_val = keras.utils.to_categorical(y_val, num_classes)

# Train the model
batch_size = 32
epochs = 10
# model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val))


In [135]:
X_train.shape

(4010, 160, 75, 1)

In [117]:
y_train.shape

(4010, 6, 62)

In [134]:
y_train.shape

(4010, 6, 62)

In [124]:
y_val.shape

(1003, 6, 62)

In [116]:
model.output_shape

(None, 6, 62)

In [107]:
log_dir = "logs"  # Specify the directory path for logs

In [108]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [128]:
model.fit(X_train, y_train.reshape(4010,6,62), batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val.reshape(1003,6,62)), callbacks=[tensorboard_callback])

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "D:\anaconda\envs\captcha\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "D:\anaconda\envs\captcha\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "D:\anaconda\envs\captcha\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "D:\anaconda\envs\captcha\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
      app.start()
    File "D:\anaconda\envs\captcha\lib\site-packages\ipykernel\kernelapp.py", line 725, in start
      self.io_loop.start()
    File "D:\anaconda\envs\captcha\lib\site-packages\tornado\platform\asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "D:\anaconda\envs\captc

In [35]:
from tensorboard import program
tb = program.TensorBoard()
tb.configure(argv=[None, '--logdir', log_dir])
url = tb.launch()
print(f"TensorBoard is running at {url}")

TensorBoard is running at http://localhost:6006/


In [36]:
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

Num GPUs Available:  0


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

In [7]:
!tensorboard dev upload --logdir \
    'logs'

^C


In [27]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

[]


In [30]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only allocate a specific amount of GPU memory
        tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Virtual devices must be set before GPUs have been initialized
        print(e)


In [32]:
gpus

[]

In [168]:
timage = keras.preprocessing.image.load_img("./preprocessed_captchas/8sxnXu/8sxnXu_1.png", color_mode='grayscale', target_size=input_shape[:2])
timage = keras.preprocessing.image.img_to_array(timage)

In [169]:
timage/=255

In [170]:
timage = np.array(timage)

In [171]:
timage.shape

(160, 75, 1)

In [172]:
X_train[0].shape

(160, 75, 1)

In [173]:
result.shape

(75, 160, 3)

In [177]:
image = keras.preprocessing.image.load_img("./preprocessed_captchas/HuayFn/HuayFn_1.png", color_mode='grayscale', target_size=input_shape[:2])
image = keras.preprocessing.image.img_to_array(image)
image /= 255.0

print(image.shape)
image = np.array([image])

(160, 75, 1)


In [178]:
predictions = model.predict([image])

1/1 [==============================] - 0s 216ms/step


In [179]:
predictions.shape

(1, 6, 62)

In [41]:

# Get the predicted labels
predicted_labels = tf.argmax(tf.squeeze(predictions), axis=-1)

# Convert the predicted labels to characters
characters = "0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ"
predicted_characters = [characters[label] for label in predicted_labels]

# Print the predicted characters
print(predicted_characters)

['0', 'u', 'a', 'y', 'F', 'n']


In [188]:
predicted_labels

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([ 0, 30, 10, 34, 41, 23], dtype=int64)>

In [189]:
# Use GPU for training

In [190]:
predictions.shape

(1, 6, 62)

In [191]:
tf.argmax(predictions[0], axis=-1)

<tf.Tensor: shape=(6,), dtype=int64, numpy=array([10,  7, 43, 29, 40, 36], dtype=int64)>